## Benchmark Nixtla implementation single-process on M5

In [1]:
!pip install hierarchicalforecast
!pip install -U numba statsforecast datasetsforecast

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
import numpy as np
import pandas as pd

#obtain hierarchical data
from datasetsforecast.hierarchical import HierarchicalData

# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut

from timeit import default_timer as timer

ROOT = "/data/cmu/large-scale-hts-reconciliation/"
data_dir = ROOT + "notebooks/"

/home/linuxbrew/.linuxbrew/opt/python@3.9/lib/python3.9/site-packages/statsforecast/core.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In this example we will use the `TourismSmall` dataset. The following cell gets the time series for the different levels in the hierarchy, the summing matrix `S` which recovers the full dataset from the bottom level hierarchy and the indices of each hierarchy denoted by `tags`.

In [3]:
import pickle

tags = pickle.load(open(data_dir + 'm5_prediction_raw/tags.pkl', 'rb'))
S_df = pickle.load(open(data_dir + 'm5_prediction_raw/nixtla_s.pkl', 'rb'))
Y_df = pd.read_csv(data_dir + 'm5_prediction_raw/nixtla_y_gt.csv')
Y_hat_df = pd.read_csv(data_dir + 'm5_prediction_raw/nixtla_y_hat.csv')

#Y_df, S_df, tags = HierarchicalData.load('./data', 'TourismSmall')

start = pd.to_datetime("2016-01-01")
    
Y_df['ds'] = pd.to_datetime(Y_df['ds'].apply(lambda x: start + pd.Timedelta(days = int(x[2:]))))

In [4]:
Y_df['ds']

0         2021-01-16
1         2021-01-17
2         2021-01-18
3         2021-01-19
4         2021-01-20
             ...    
3048995   2021-04-21
3048996   2021-04-22
3048997   2021-04-23
3048998   2021-04-24
3048999   2021-04-25
Name: ds, Length: 3049000, dtype: datetime64[ns]

In [5]:
Y_df.head()

,unique_id,ds,y
0,HOBBIES_1_001_CA_1,2021-01-16,4.0
1,HOBBIES_1_001_CA_1,2021-01-17,0.0
2,HOBBIES_1_001_CA_1,2021-01-18,0.0
3,HOBBIES_1_001_CA_1,2021-01-19,0.0
4,HOBBIES_1_001_CA_1,2021-01-20,0.0


In [6]:
S_df.iloc[:5, :5]

,HOBBIES_1_001_CA_1,HOBBIES_1_002_CA_1,HOBBIES_1_003_CA_1,HOBBIES_1_004_CA_1,HOBBIES_1_005_CA_1
HOBBIES,1,1,1,1,1
HOUSEHOLD,0,0,0,0,0
FOODS,0,0,0,0,0
HOBBIES_1,1,1,1,1,1
HOBBIES_2,0,0,0,0,0


In [7]:
tags

{'level_1': array(['HOBBIES', 'HOUSEHOLD', 'FOODS'], dtype=object),
 'level_2': array(['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1',
        'FOODS_2', 'FOODS_3'], dtype=object),
 'level_3': array(['HOBBIES_1_001', 'HOBBIES_1_002', 'HOBBIES_1_003', ...,
        'FOODS_3_825', 'FOODS_3_826', 'FOODS_3_827'], dtype=object),
 'level_4': array(['HOBBIES_1_001_CA_1', 'HOBBIES_1_002_CA_1', 'HOBBIES_1_003_CA_1',
        ..., 'FOODS_3_825_WI_3', 'FOODS_3_826_WI_3', 'FOODS_3_827_WI_3'],
       dtype=object)}

In [8]:
Y_hat_df['ds'] = pd.to_datetime(Y_hat_df['ds'].apply(lambda x: start + pd.Timedelta(days = int(x[2:]))))
Y_hat_df.head()

,unique_id,ds,prophet
0,HOBBIES_1_001_CA_1,2021-01-16,0.641144
1,HOBBIES_1_001_CA_1,2021-01-17,0.855605
2,HOBBIES_1_001_CA_1,2021-01-18,0.713611
3,HOBBIES_1_001_CA_1,2021-01-19,1.213304
4,HOBBIES_1_001_CA_1,2021-01-20,1.142696


The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. The used methods to make the forecasts coherent are:
- `BottomUp`: The reconciliation of the method is a simple addition to the upper levels.
- `TopDown`: The second method constrains the base-level predictions to the top-most aggregate-level serie and then distributes it to the disaggregate series through the use of proportions. 
- `MiddleOut`: Anchors the base predictions in a middle level.

In [11]:
date = '2021-01-16'

Y_hat = Y_hat_df.loc[Y_hat_df['ds'] == date]
Y = Y_df.loc[Y_df['ds'] == date]

reconcilers = [
    BottomUp(),
    #TopDown(method='forecast_proportions'),
    #MiddleOut(middle_level='Country/Purpose/State', 
    #          top_down_method='forecast_proportions')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

start = timer()
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat, Y_df=Y, 
                          S=S_df, tags=tags)
end = timer()
print(end - start)

Exception: Check `S_df`, `Y_hat_df` series difference, S\Y_hat=33549, Y_hat\S=30490

In [13]:
Y

,unique_id,ds,y
0,HOBBIES_1_001_CA_1,2021-01-16,4.0
100,HOBBIES_1_002_CA_1,2021-01-16,0.0
200,HOBBIES_1_003_CA_1,2021-01-16,1.0
300,HOBBIES_1_004_CA_1,2021-01-16,2.0
400,HOBBIES_1_005_CA_1,2021-01-16,5.0
...,...,...,...
3048500,FOODS_3_823_WI_3,2021-01-16,0.0
3048600,FOODS_3_824_WI_3,2021-01-16,0.0
3048700,FOODS_3_825_WI_3,2021-01-16,0.0
3048800,FOODS_3_826_WI_3,2021-01-16,1.0


In [ ]:
S_df